<a href="https://colab.research.google.com/github/Ramathabathe19/github-actions-python-mock/blob/main/lerato_revised_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import json
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta

# Configuration
LOOKBACK_WINDOW = 10  # Number of periods for rolling calculations
Z_SCORE_THRESHOLD = 2  # Threshold for mean reversion signals
MOMENTUM_THRESHOLD = 0.0001  # Threshold for momentum signals (price change)
SHORT_TERM_MA = 5  # Short-term moving average window
LONG_TERM_MA = 10  # Long-term moving average window
SCALPING_THRESHOLD = 0.00005  # Threshold for scalping signals
BREAKOUT_THRESHOLD = 0.0002  # Threshold for breakout signals

# Fetch Historical Data from Yahoo Finance
def fetch_historical_data(currency_pairs, start_date, end_date, interval="1d"):
    """
    Fetch historical data for currency pairs using yfinance.
    """
    data = {}
    for pair in currency_pairs:
        ticker = f"{pair}=X"  # Yahoo Finance uses "=X" suffix for forex pairs
        print(f"Fetching data for {ticker}...")
        df = yf.download(ticker, start=start_date, end=end_date, interval=interval)
        if df.empty:
            print(f"No data found for {ticker}. Skipping.")
            continue
        data[pair] = df
    return data

# Mean Reversion Logic
def calculate_mean_reversion(historical_data, lookback_window=LOOKBACK_WINDOW, z_score_threshold=Z_SCORE_THRESHOLD):
    """
    Calculate mean reversion signals based on historical close prices.
    """
    results = []
    for currency_pair, df in historical_data.items():
        prices = df["Close"].values
        timestamps = df.index
        for i in range(lookback_window, len(prices)):
            current_price = float(prices[i])  # Convert to native Python float
            lookback_prices = prices[i - lookback_window:i]
            # Calculate rolling statistics
            rolling_mean = float(np.mean(lookback_prices))  # Convert to native Python float
            rolling_std = float(np.std(lookback_prices))    # Convert to native Python float
            z_score = (current_price - rolling_mean) / rolling_std if rolling_std != 0 else 0
            # Generate trading signals
            signal = 0
            if z_score > z_score_threshold:
                signal = -1  # Sell signal
            elif z_score < -z_score_threshold:
                signal = 1  # Buy signal
            # Append result for output
            results.append({
                "timestamp": timestamps[i].isoformat(),
                "currency_pair": currency_pair,
                "price": current_price,
                "rolling_mean": rolling_mean,
                "z_score": z_score,
                "signal": signal
            })
    return results

# Momentum Strategy Logic
def calculate_momentum(historical_data, lookback_window=LOOKBACK_WINDOW, momentum_threshold=MOMENTUM_THRESHOLD):
    """
    Calculate momentum signals based on price changes over the lookback window.
    """
    results = []
    for currency_pair, df in historical_data.items():
        prices = df["Close"].values
        timestamps = df.index
        for i in range(lookback_window, len(prices)):
            current_price = float(prices[i])  # Convert to native Python float
            lookback_prices = prices[i - lookback_window:i]
            # Calculate momentum (price change over the lookback window)
            momentum = current_price - lookback_prices[0]
            # Generate trading signals
            signal = 0
            if momentum > momentum_threshold:
                signal = 1  # Buy signal (upward trend)
            elif momentum < -momentum_threshold:
                signal = -1  # Sell signal (downward trend)
            # Append result for output
            results.append({
                "timestamp": timestamps[i].isoformat(),
                "currency_pair": currency_pair,
                "price": current_price,
                "momentum": momentum,
                "signal": signal
            })
    return results

# Trend Following Logic
def calculate_trend_following(historical_data, short_term_ma=SHORT_TERM_MA, long_term_ma=LONG_TERM_MA):
    """
    Calculate trend following signals based on moving average crossovers.
    """
    results = []
    for currency_pair, df in historical_data.items():
        prices = df["Close"].values
        timestamps = df.index
        for i in range(long_term_ma, len(prices)):
            current_price = float(prices[i])  # Convert to native Python float
            short_ma = float(np.mean(prices[i - short_term_ma:i]))
            long_ma = float(np.mean(prices[i - long_term_ma:i]))
            # Generate trading signals
            signal = 0
            if short_ma > long_ma:
                signal = 1  # Buy signal (uptrend)
            elif short_ma < long_ma:
                signal = -1  # Sell signal (downtrend)
            # Append result for output
            results.append({
                "timestamp": timestamps[i].isoformat(),
                "currency_pair": currency_pair,
                "price": current_price,
                "short_ma": short_ma,
                "long_ma": long_ma,
                "signal": signal
            })
    return results

# Scalping Strategy Logic
def calculate_scalping(historical_data, scalping_threshold=SCALPING_THRESHOLD):
    """
    Calculate scalping signals based on small price deviations.
    """
    results = []
    for currency_pair, df in historical_data.items():
        prices = df["Close"].values
        timestamps = df.index
        previous_price = None
        for i in range(len(prices)):
            current_price = float(prices[i])  # Convert to native Python float
            # Calculate deviation from the previous price
            deviation = current_price - previous_price if previous_price is not None else 0
            # Generate trading signals
            signal = 0
            if deviation > scalping_threshold:
                signal = 1  # Buy signal
            elif deviation < -scalping_threshold:
                signal = -1  # Sell signal
            # Update previous price
            previous_price = current_price
            # Append result for output
            results.append({
                "timestamp": timestamps[i].isoformat(),
                "currency_pair": currency_pair,
                "price": current_price,
                "deviation": deviation,
                "signal": signal
            })
    return results

# Breakout Strategy Logic
def calculate_breakout(historical_data, breakout_threshold=BREAKOUT_THRESHOLD):
    """
    Calculate breakout signals based on price breaking key levels.
    """
    results = []
    for currency_pair, df in historical_data.items():
        prices = df["Close"].values
        timestamps = df.index
        high_level = low_level = prices[0]
        for i in range(len(prices)):
            current_price = float(prices[i])  # Convert to native Python float
            # Update high and low levels
            high_level = max(high_level, current_price)
            low_level = min(low_level, current_price)
            # Generate trading signals
            signal = 0
            if current_price > high_level + breakout_threshold:
                signal = 1  # Buy signal
            elif current_price < low_level - breakout_threshold:
                signal = -1  # Sell signal
            # Append result for output
            results.append({
                "timestamp": timestamps[i].isoformat(),
                "currency_pair": currency_pair,
                "price": current_price,
                "high_level": high_level,
                "low_level": low_level,
                "signal": signal
            })
    return results

# Trading Simulation
def simulate_trading(signals):
    """
    Simulate trading based on generated signals.
    """
    position = {}  # Track positions for each currency pair
    pnl = {}       # Track profit/loss for each currency pair
    cumulative_pnl = []
    for signal in signals:
        currency_pair = signal["currency_pair"]
        price = signal["price"]
        signal_value = signal["signal"]
        # Initialize position and PnL for the currency pair
        if currency_pair not in position:
            position[currency_pair] = 0
            pnl[currency_pair] = 0
        # Update position based on signal
        if signal_value == 1 and position[currency_pair] <= 0:  # Buy signal
            position[currency_pair] += 1
        elif signal_value == -1 and position[currency_pair] >= 0:  # Sell signal
            position[currency_pair] -= 1
        # Update PnL based on price change
        if position[currency_pair] != 0:
            previous_price = signals[signals.index(signal) - 1]["price"] if signals.index(signal) > 0 else price
            pnl[currency_pair] += position[currency_pair] * (price - previous_price)
        # Append cumulative PnL
        cumulative_pnl.append({
            "timestamp": signal["timestamp"],
            "currency_pair": currency_pair,
            "cumulative_pnl": pnl[currency_pair]
        })
    return cumulative_pnl

# Extract Buy/Sell Points
def extract_buy_sell_points(signals):
    """
    Extract buy and sell points from the signals.

    Args:
        signals (list): List of signal dictionaries containing 'timestamp', 'currency_pair', 'price', and 'signal'.

    Returns:
        dict: A dictionary where keys are currency pairs and values are lists of buy/sell points.
    """
    buy_sell_points = {}
    for signal in signals:
        currency_pair = signal["currency_pair"]
        timestamp = signal["timestamp"]
        price = signal["price"]
        signal_value = signal["signal"]

        # Initialize entry for the currency pair if not already present
        if currency_pair not in buy_sell_points:
            buy_sell_points[currency_pair] = {"buy": [], "sell": []}

        # Append buy or sell points based on the signal
        if signal_value == 1:  # Buy signal
            buy_sell_points[currency_pair]["buy"].append({"timestamp": timestamp, "price": price})
        elif signal_value == -1:  # Sell signal
            buy_sell_points[currency_pair]["sell"].append({"timestamp": timestamp, "price": price})

    return buy_sell_points

# Main Execution
if __name__ == "__main__":
    # Step 1: Define currency pairs and fetch historical data
    currency_pairs = ["EURUSD", "USDJPY", "GBPUSD", "EURGBP", "USDCAD"]
    start_date = (datetime.now() - timedelta(days=365)).strftime("%Y-%m-%d")  # Last year
    end_date = datetime.now().strftime("%Y-%m-%d")  # Today
    historical_data = fetch_historical_data(currency_pairs, start_date, end_date)

    # Step 2: Calculate signals for all five strategies
    mean_reversion_signals = calculate_mean_reversion(historical_data)
    momentum_signals = calculate_momentum(historical_data)
    trend_following_signals = calculate_trend_following(historical_data)
    scalping_signals = calculate_scalping(historical_data)
    breakout_signals = calculate_breakout(historical_data)

    # Step 3: Simulate trading for all five strategies
    mean_reversion_results = simulate_trading(mean_reversion_signals)
    momentum_results = simulate_trading(momentum_signals)
    trend_following_results = simulate_trading(trend_following_signals)
    scalping_results = simulate_trading(scalping_signals)
    breakout_results = simulate_trading(breakout_signals)

    # Step 4: Save trading results to JSON files
    strategies = {
        "mean_reversion": mean_reversion_results,
        "momentum": momentum_results,
        "trend_following": trend_following_results,
        "scalping": scalping_results,
        "breakout": breakout_results
    }
    for strategy_name, results in strategies.items():
        with open(f"{strategy_name}_results.json", "w") as json_file:
            json.dump(results, json_file, indent=4)
        print(f"Results for {strategy_name.capitalize()} saved to '{strategy_name}_results.json'.")

    # Step 5: Extract and save buy/sell points for each strategy
    mean_reversion_buy_sell = extract_buy_sell_points(mean_reversion_signals)
    momentum_buy_sell = extract_buy_sell_points(momentum_signals)
    trend_following_buy_sell = extract_buy_sell_points(trend_following_signals)
    scalping_buy_sell = extract_buy_sell_points(scalping_signals)
    breakout_buy_sell = extract_buy_sell_points(breakout_signals)

    strategy_buy_sell_points = {
        "mean_reversion": mean_reversion_buy_sell,
        "momentum": momentum_buy_sell,
        "trend_following": trend_following_buy_sell,
        "scalping": scalping_buy_sell,
        "breakout": breakout_buy_sell
    }

    for strategy_name, buy_sell_data in strategy_buy_sell_points.items():
        with open(f"{strategy_name}_buy_sell_points.json", "w") as json_file:
            json.dump(buy_sell_data, json_file, indent=4)
        print(f"Buy/Sell points for {strategy_name.capitalize()} saved to '{strategy_name}_buy_sell_points.json'.")

    # Step 6: Recommendation
    final_pnl = {}
    for pair in currency_pairs:
        final_pnl[pair] = {
            "mean_reversion": next((r["cumulative_pnl"] for r in mean_reversion_results if r["currency_pair"] == pair), 0),
            "momentum": next((r["cumulative_pnl"] for r in momentum_results if r["currency_pair"] == pair), 0),
            "trend_following": next((r["cumulative_pnl"] for r in trend_following_results if r["currency_pair"] == pair), 0),
            "scalping": next((r["cumulative_pnl"] for r in scalping_results if r["currency_pair"] == pair), 0),
            "breakout": next((r["cumulative_pnl"] for r in breakout_results if r["currency_pair"] == pair), 0)
        }

    # Display the best-performing strategy for each currency pair
    print("\n--- Best Performing Strategy for Each Currency Pair ---")
    for pair, pnl_by_strategy in final_pnl.items():
        best_strategy = max(pnl_by_strategy, key=pnl_by_strategy.get)
        best_pnl = pnl_by_strategy[best_strategy]
        print(f"{pair}: Best Strategy = {best_strategy.capitalize()} with PnL = {best_pnl}")

    # Save recommendations to JSON file
    recommendations = {
        "final_pnl": final_pnl,
        "best_strategies": {pair: max(pnl_by_strategy, key=pnl_by_strategy.get) for pair, pnl_by_strategy in final_pnl.items()}
    }
    with open("strategy_recommendations.json", "w") as json_file:
        json.dump(recommendations, json_file, indent=4)
    print("\nStrategy recommendations saved to 'strategy_recommendations.json'.")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Fetching data for EURUSD=X...
Fetching data for USDJPY=X...
Fetching data for GBPUSD=X...
Fetching data for EURGBP=X...
Fetching data for USDCAD=X...



<ipython-input-3-5363c4c1f9e1>:42: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  current_price = float(prices[i])  # Convert to native Python float
<ipython-input-3-5363c4c1f9e1>:75: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  current_price = float(prices[i])  # Convert to native Python float
<ipython-input-3-5363c4c1f9e1>:105: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  current_price = float(prices[i])  # Convert to native Python float
<ipython-input-3-5363c4

Results for Mean_reversion saved to 'mean_reversion_results.json'.
Results for Momentum saved to 'momentum_results.json'.
Results for Trend_following saved to 'trend_following_results.json'.
Results for Scalping saved to 'scalping_results.json'.
Results for Breakout saved to 'breakout_results.json'.
Buy/Sell points for Mean_reversion saved to 'mean_reversion_buy_sell_points.json'.
Buy/Sell points for Momentum saved to 'momentum_buy_sell_points.json'.
Buy/Sell points for Trend_following saved to 'trend_following_buy_sell_points.json'.
Buy/Sell points for Scalping saved to 'scalping_buy_sell_points.json'.
Buy/Sell points for Breakout saved to 'breakout_buy_sell_points.json'.

--- Best Performing Strategy for Each Currency Pair ---
EURUSD: Best Strategy = Mean_reversion with PnL = 0
USDJPY: Best Strategy = Momentum with PnL = 150.31243085861206
GBPUSD: Best Strategy = Momentum with PnL = 146.68527793884277
EURGBP: Best Strategy = Momentum with PnL = 0.44183027744293213
USDCAD: Best Strate